# plumbum Tutorial

This notebook walks through the core features of `pdum.plumbum`, starting with synchronous pipelines and progressing to asynchronous iterator workflows.

## Quick Start

Use the `@pb` decorator to turn functions into operators, compose them with `|`, and execute pipelines with `>>`.

In [ ]:
from pdum.plumbum import pb


@pb
def add(x, amount):
    return x + amount


@pb
def multiply(x, factor):
    return x * factor


pipeline = add(3) | multiply(2) | add(5)
10 >> pipeline

### Partial Application

Operators can be partially applied across multiple calls before execution.

In [ ]:
@pb
def add_three(x, a, b, c):
    return x + a + b + c


op = add_three(1)
op = op(2)
op = op(3)
5 >> op

### Plain Functions as Operators

Plain callables are promoted to operators automatically when combined with `|`.

In [ ]:
from functools import partial


def plain_increment(x):
    return x + 3


def plain_add(x, n):
    return x + n


pipeline = multiply(2) | plain_increment | partial(plain_add, n=4)
6 >> pipeline

### Data Type Flexibility

plumbum threads any Python object—numbers, strings, dictionaries, custom classes—through your functions.

In [ ]:
"hello" >> pb(str.upper) >> pb(lambda s: s + "!")
{"a": 1} >> pb(lambda d: {**d, "b": 2})


class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __repr__(self):
        return f"Point({self.x}, {self.y})"


@pb
def translate(point, dx, dy):
    return Point(point.x + dx, point.y + dy)


Point(1, 2) >> translate(3, 4)

### Debugging Pipelines

Wrap helpers such as `print` with `pb` to inspect intermediate values.

In [ ]:
result = 10 >> add(5) >> pb(print) >> multiply(2) >> pb(print)

result

## Asynchronous Pipelines

Use `apb` to create async-aware operators. Pipelines return awaitable results that you resolve with `await` or `asyncio.run`.

In [ ]:
import asyncio
from pdum.plumbum import apb


@apb
async def fetch_amount(value):
    await asyncio.sleep(0)
    return value + 5


@apb
async def scale(value, factor):
    await asyncio.sleep(0)
    return value * factor


async def run_async_pipeline():
    pipeline = fetch_amount | scale(2)
    return await (10 >> pipeline)


asyncio.run(run_async_pipeline())

### Mixing Sync and Async Operators

`apb` wraps synchronous functions automatically so you can mix them inside async pipelines.

In [ ]:
@apb
async def async_double(x):
    await asyncio.sleep(0)
    return x * 2


@apb
def add_sync(x, delta):
    return x + delta


async def run_mixed_pipeline():
    pipeline = async_double | add_sync(3) | scale(2)
    return await (5 >> pipeline)


asyncio.run(run_mixed_pipeline())